This file dont have to be executed, it exist only for explaining how i achieved the file "kills.csv"<br>
The file "session.csv" is required
# Library

In [1]:
## Pandas for reading 
import pandas as pd
## Clear Output
from IPython.display import clear_output
## For transforming millisecond to date
from datetime import datetime
## For nan
import numpy as np
import math

# DataSet

In [2]:
kill_db = pd.read_csv("../../data/server_table/plan_kills.csv", index_col = "id")
## Removing the column "server_uuid"
kill_db.drop(columns = ["server_uuid"], inplace = True)
date = []
for i in range(kill_db.__len__()):
    ## Transforming ms time to human
    date.append(pd.Timestamp(datetime.fromtimestamp((kill_db.iloc[i].date/1000.0).astype(int))))
## Replacing
kill_db.date = date

# Analyze

## Dictionary
Lets count every kill - death and what weapon was used.

In [3]:
## First of all, we have to create our dictionary that will contains every value we want.
## Count every killer
killer = {uuid : kills for uuid, kills in kill_db["killer_uuid"].value_counts().iteritems()}
## Count every victim
victim = {uuid : death for uuid, death in kill_db["victim_uuid"].value_counts().iteritems()}
## Before doing for every item, we have to replace "air" with "hand"
kill_db.weapon = kill_db.weapon.replace("Air", "Hand")
weapon = {weapon : kill for weapon, kill in kill_db["weapon"].value_counts().iteritems()}

## Location
Since we dont have any coordinates, we cannot say the excact location.<br>
I found another way, the only downside is that is not precise and we have to approximate: <br>
We can take the coordinates where the guy joined before he died. <br>
I know it's not precise, but it's our only way

### Variable

In [4]:
## Read our session
session = pd.read_pickle("../../data/ufficial/pickle/session.pkl")

In [5]:
## From an index, get the coordinates
def get_position(index):
    ## Get the session
    index_session = kill_db.at[index, "session_id"]
    ## Return x-y-z
    return session.loc[index_session][["x_join", "y_join", "z_join"]]

In [6]:
## The list where we are going to store our datas
last = kill_db.index[-1]
## Add columns
kill_db[["x_death", "y_death", "z_death"]] = pd.DataFrame([[np.nan, np.nan, np.nan]], index = kill_db.index)
## Iterate
for i in kill_db.index.values:
    ## Append the position
    kill_db.loc[i, ["x_death", "y_death", "z_death"]] = get_position(i).values
    if i % 500 == 0:
        clear_output()
        print(i / last * 100)

99.98018410765434


# Uuid to Name

In [7]:
## Import uuid
uuid_db = pd.read_pickle("../../data/ufficial/pickle/uuid.pkl")
## Make both time the same exponent
kill_db["date"] = kill_db["date"].astype(np.int64) / 1E5
uuid_db["time"] = uuid_db["time"] * 1E1

#### Lets find everything we want
so, i dunno why but, for some reasons, some uuid have multiple names. that's why of the iloc[0]

In [25]:
kill_db["killer_name"] = kill_db.apply(lambda x : uuid_db[uuid_db["uuid"] == x["killer_uuid"]].iloc[0].nickname,\
                                       axis=1)

In [26]:
kill_db["victim_name"] = kill_db.apply(lambda x : uuid_db[uuid_db["uuid"] == x["victim_uuid"]].iloc[0].nickname,\
                                       axis=1)

# Export csv

In [27]:
kill_db.to_csv("../../data/ufficial/csv/kills.csv")
kill_db.to_pickle("../../data/ufficial/pickle/kills.pkl")